<a href="https://colab.research.google.com/github/MHDBST/BERT_examples/blob/master/MaskLM_document_TPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
import json
import os
import pprint
import random
import string
import sys
import tensorflow as tf
import pandas as pd
!pip install bert-tensorflow

assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is', TPU_ADDRESS)

from google.colab import auth
auth.authenticate_user()
with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
  # Now credentials are set for all future sessions on this TPU.

('TPU address is', 'grpc://10.80.80.106:8470')
TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 11638007704623703138),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 106975361678022239),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 7977074008086956635),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 1708813072471818492),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 4239857053124479385),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 3604834312952548474),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 18195907398084105712),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 18339480919885293788),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:6, TPU, 17179869184, 9306903

In [2]:
import sys

!test -d bert_repo || git clone https://github.com/google-research/bert bert_repo
if not 'bert_repo' in sys.path:
  sys.path += ['bert_repo']

# import python modules defined by BERT
from bert import modeling
import optimization
# import run_classifier
# from bert import run_classifier_with_tfhub
# import tokenization

# import tfhub 
import tensorflow_hub as hub



W0517 18:01:23.218550 140065797625728 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


In [3]:
BUCKET = 'bert_example' #@param {type:"string"}
assert BUCKET, 'Must specify an existing GCS bucket name'
OUTPUT_DIR = 'gs://{}/mask_lm/models/V2_augment/doc_level/v2/smallBERT-docLevel-seq512'.format(BUCKET)
tf.gfile.MakeDirs(OUTPUT_DIR)

print('***** Model output directory: {} *****'.format(OUTPUT_DIR))

# Available pretrained model checkpoints:
#   uncased_L-12_H-768_A-12: uncased BERT base model
#   uncased_L-24_H-1024_A-16: uncased BERT large model
#   cased_L-12_H-768_A-12: cased BERT large model
BERT_MODEL = 'uncased_L-12_H-768_A-12' #@param {type:"string"}
BERT_MODEL_HUB = 'https://tfhub.dev/google/bert_' + BERT_MODEL + '/1'
BERT_PRETRAINED_DIR = 'gs://cloud-tpu-checkpoints/bert/' + BERT_MODEL
print('***** BERT pretrained directory: {} *****'.format(BERT_PRETRAINED_DIR))
!gsutil ls $BERT_PRETRAINED_DIR





***** Model output directory: gs://bert_example/mask_lm/models/V2_augment/doc_level/v2/smallBERT-docLevel-seq512 *****
***** BERT pretrained directory: gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12 *****
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/bert_config.json
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/bert_model.ckpt.index
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/bert_model.ckpt.meta
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/checkpoint
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/vocab.txt


In [0]:
# import modeling
# import run_classifier
import tokenization

# coding=utf-8
# Copyright 2018 The Google AI Language Team Authors.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""Functions and classes related to optimization (weight updates)."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import re
import tensorflow as tf


def create_optimizer(loss, init_lr, num_train_steps, num_warmup_steps, use_tpu):
  """Creates an optimizer training op."""
  global_step = tf.train.get_or_create_global_step()

  learning_rate = tf.constant(value=init_lr, shape=[], dtype=tf.float32)

  # Implements linear decay of the learning rate.
  learning_rate = tf.train.polynomial_decay(
      learning_rate,
      global_step,
      num_train_steps,
      end_learning_rate=0.0,
      power=1.0,
      cycle=False)

  # Implements linear warmup. I.e., if global_step < num_warmup_steps, the
  # learning rate will be `global_step/num_warmup_steps * init_lr`.
  if num_warmup_steps:
    global_steps_int = tf.cast(global_step, tf.int32)
    warmup_steps_int = tf.constant(num_warmup_steps, dtype=tf.int32)

    global_steps_float = tf.cast(global_steps_int, tf.float32)
    warmup_steps_float = tf.cast(warmup_steps_int, tf.float32)

    warmup_percent_done = global_steps_float / warmup_steps_float
    warmup_learning_rate = init_lr * warmup_percent_done

    is_warmup = tf.cast(global_steps_int < warmup_steps_int, tf.float32)
    learning_rate = (
        (1.0 - is_warmup) * learning_rate + is_warmup * warmup_learning_rate)

  # It is recommended that you use this optimizer for fine tuning, since this
  # is how the model was trained (note that the Adam m/v variables are NOT
  # loaded from init_checkpoint.)
  optimizer = AdamWeightDecayOptimizer(
      learning_rate=learning_rate,
      weight_decay_rate=0.01,
      beta_1=0.9,
      beta_2=0.999,
      epsilon=1e-6,
      exclude_from_weight_decay=["LayerNorm", "layer_norm", "bias"])

  if use_tpu:
    optimizer = tf.contrib.tpu.CrossShardOptimizer(optimizer)

  pre_tvars = tf.trainable_variables()
#   print('all trainable variables: >>',pre_tvars)
  tvars = pre_tvars
  tvars = [item for item in pre_tvars if not '/layer_0/'  in item.name and not '/layer_1/'  in item.name and not '/layer_2/'  in item.name
          and not '/layer_3/'  in item.name and not '/layer_4/'  in item.name and not '/layer_5/'  in item.name and not '/layer_6/'  in item.name 
          and not '/layer_7/'  in item.name and not '/layer_8/'  in item.name and not '/layer_9/' in item.name]
#   and not '/layer_10/'  in item.name 
#           and not '/layer_11/'  in item.name ]
  print('excluded trainable variables: >>',tvars)
  grads = tf.gradients(loss, tvars)

  # This is how the model was pre-trained.
  (grads, _) = tf.clip_by_global_norm(grads, clip_norm=1.0)

  train_op = optimizer.apply_gradients(
      zip(grads, tvars), global_step=global_step)

  # Normally the global step update is done inside of `apply_gradients`.
  # However, `AdamWeightDecayOptimizer` doesn't do this. But if you use
  # a different optimizer, you should probably take this line out.
  new_global_step = global_step + 1
  train_op = tf.group(train_op, [global_step.assign(new_global_step)])
  return train_op


class AdamWeightDecayOptimizer(tf.train.Optimizer):
  """A basic Adam optimizer that includes "correct" L2 weight decay."""

  def __init__(self,
               learning_rate,
               weight_decay_rate=0.0,
               beta_1=0.9,
               beta_2=0.999,
               epsilon=1e-6,
               exclude_from_weight_decay=None,
               name="AdamWeightDecayOptimizer"):
    """Constructs a AdamWeightDecayOptimizer."""
    super(AdamWeightDecayOptimizer, self).__init__(False, name)

    self.learning_rate = learning_rate
    self.weight_decay_rate = weight_decay_rate
    self.beta_1 = beta_1
    self.beta_2 = beta_2
    self.epsilon = epsilon
    self.exclude_from_weight_decay = exclude_from_weight_decay

  def apply_gradients(self, grads_and_vars, global_step=None, name=None):
    """See base class."""
    assignments = []
    for (grad, param) in grads_and_vars:
      if grad is None or param is None:
        continue

      param_name = self._get_variable_name(param.name)

      m = tf.get_variable(
          name=param_name + "/adam_m",
          shape=param.shape.as_list(),
          dtype=tf.float32,
          trainable=False,
          initializer=tf.zeros_initializer())
      v = tf.get_variable(
          name=param_name + "/adam_v",
          shape=param.shape.as_list(),
          dtype=tf.float32,
          trainable=False,
          initializer=tf.zeros_initializer())

      # Standard Adam update.
      next_m = (
          tf.multiply(self.beta_1, m) + tf.multiply(1.0 - self.beta_1, grad))
      next_v = (
          tf.multiply(self.beta_2, v) + tf.multiply(1.0 - self.beta_2,
                                                    tf.square(grad)))

      update = next_m / (tf.sqrt(next_v) + self.epsilon)

      # Just adding the square of the weights to the loss function is *not*
      # the correct way of using L2 regularization/weight decay with Adam,
      # since that will interact with the m and v parameters in strange ways.
      #
      # Instead we want ot decay the weights in a manner that doesn't interact
      # with the m/v parameters. This is equivalent to adding the square
      # of the weights to the loss with plain (non-momentum) SGD.
      if self._do_use_weight_decay(param_name):
        update += self.weight_decay_rate * param

      update_with_lr = self.learning_rate * update

      next_param = param - update_with_lr

      assignments.extend(
          [param.assign(next_param),
           m.assign(next_m),
           v.assign(next_v)])
    return tf.group(*assignments, name=name)

  def _do_use_weight_decay(self, param_name):
    """Whether to use L2 weight decay for `param_name`."""
    if not self.weight_decay_rate:
      return False
    if self.exclude_from_weight_decay:
      for r in self.exclude_from_weight_decay:
        if re.search(r, param_name) is not None:
          return False
    return True

  def _get_variable_name(self, param_name):
    """Get the variable name from the tensor name."""
    m = re.match("^(.*):\\d+$", param_name)
    if m is not None:
      param_name = m.group(1)
    return param_name



In [0]:
TRAIN_BATCH_SIZE = 8
EVAL_BATCH_SIZE = 8
PREDICT_BATCH_SIZE = 8
LEARNING_RATE = 2e-4
NUM_TRAIN_EPOCHS = 10.0  ## Activate if ** is Not ACTIVATED
MAX_SEQ_LENGTH = 512
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 50
SAVE_SUMMARY_STEPS = 20



In [6]:
import pandas as pd


data_train = pd.read_csv('/content/mask_lm_3Dec_7Dec_reindex_train_v3.csv', encoding='latin-1')[:-1]
data_augment = pd.read_csv('/content/mask_lm_postprocess.csv', encoding='latin-1')[:-1]
data_dev = pd.read_csv('/content/mask_lm_3Dec_7Dec_reindex_dev_v3.csv', encoding='latin-1')[:-1]
data_test = pd.read_csv('/content/mask_lm_3Dec_7Dec_reindex_random_test_v3.csv', encoding='latin-1')[:-1]
data_test_fixed = pd.read_csv('/content/mask_lm_3Dec_7Dec_reindex_fixed_test_v3.csv', encoding='latin-1')[:-1]

# data_train = pd.read_csv('/content/masked_lm_train.csv', encoding='latin-1')[:200]#[:-1]
# data_dev = pd.read_csv('/content/masked_lm_dev.csv', encoding='latin-1')[:200]#[:-1]
# data_test = pd.read_csv('/content/masked_lm_random_test.csv', encoding='latin-1')[:200]#[:-1]
# data_test_fixed = pd.read_csv('/content/masked_lm_fixed_test.csv', encoding='latin-1')[:200]#[:-1]

# Load all files from a directory in a DataFrame.
def load_directory_data(df):
#   print('df length>>',len(df['DOCUMENT']))
  data = {}
  data["sentence"] = df['DOCUMENT']
  data["label"] =df["LABEL"]
#   print('data length>>',data)
  return pd.DataFrame.from_dict(data)

# Merge positive and negative examples, add a polarity column and shuffle.
def load_dataset(df,index = None):
  df_new = load_directory_data(df[:index])
#   print(df_new)
  true_df = df_new[df_new['label'] == True]
  false_df = df_new[df_new['label'] == False]
#   print('true_df>>>',len(true_df))
#   true_df["polarity"] = 1
#   false_df["polarity"] = 0
  return pd.concat([true_df, false_df]).sample(frac=1).reset_index(drop=True)


train = load_dataset(data_train)
train_augment = load_dataset(data_augment)
train = pd.concat([train, train_augment]).sample(frac=1).reset_index(drop=True)
test = load_dataset(data_test)
dev = load_dataset(data_dev)
test_fixed = load_dataset(data_test_fixed)

print('train set length: %d,dev set length: %d, test set lenght: %d,  fixed test: %d'%
      (len(train),len(dev),len(test),len(test_fixed)))
# 
# print('new train length is %d'%len(train))
# with open('/content/masked_lm_document_input.txt','w') as f:
  
#   for doc in list(train['sentence']):
#     f.write(doc.encode('utf-8'))
#     f.write('\n\n')
    
    

    




train set length: 122150,dev set length: 3479, test set lenght: 4023,  fixed test: 5038


In [7]:
DATA_COLUMN = 'sentence'
LABEL_COLUMN = 'label'
label_list = [0,1]
use_tpu = True

a = 1./len(dev[dev['label']==0])
b = 1./len(dev[dev['label']==1])
print(a/(a+b) , b/(a+b))

a = 1./len(train[train['label']==0])
b = 1./len(train[train['label']==1])
print(a/(a+b) , b/(a+b))

0.647312446105 0.352687553895
0.506958657388 0.493041342612


In [0]:
import run_classifier
train_InputExamples = train.apply(lambda x: run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

dev_InputExamples = dev.apply(lambda x: run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)


test_InputExamples = test.apply(lambda x: run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

test_InputExamples_fixed = test_fixed.apply(lambda x: run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)



## These two lines should be activated if ** is not activated
num_train_steps = int(len(train_InputExamples) / TRAIN_BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

# Setup TPU related config
tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(TPU_ADDRESS)
NUM_TPU_CORES = 8
# ITERATIONS_PER_LOOP = 100 # I don't know what it is doing just decrease it to smaller value
ITERATIONS_PER_LOOP = 500#int(len(train_InputExamples) / TRAIN_BATCH_SIZE) ## set as the number of iterations in each epoch 


# def get_run_config(output_dir):
#   return tf.contrib.tpu.RunConfig(
#     cluster=tpu_cluster_resolver,
#     model_dir=output_dir,
#     save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
#     tpu_config=tf.contrib.tpu.TPUConfig(
#         iterations_per_loop=ITERATIONS_PER_LOOP,
#         num_shards=NUM_TPU_CORES,
#         per_host_input_for_training=tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2))



In [0]:
path = 'gs://bert_example/bert/uncased_L-12_H-768_A-12/vocab_tgt.txt'
f_in = tf.gfile.GFile('gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/vocab.txt')
f_out = tf.gfile.GFile(path,'w')
lines = f_in.readlines()
lines[1] = '[TGT]\n'
lines[2] = '[tgt]\n'
lines[3] = 'TGT\n'
lines[4] = 'tgt\n'
for line in lines:
  f_out.write(line)
f_out.close()

VOCAB_FILE = os.path.join('gs://bert_example/bert/uncased_L-12_H-768_A-12', 'vocab_tgt.txt')
CONFIG_FILE = os.path.join(BERT_PRETRAINED_DIR, 'bert_config.json')
INIT_CHECKPOINT = os.path.join(BERT_PRETRAINED_DIR, 'bert_model.ckpt')
DO_LOWER_CASE = BERT_MODEL.startswith('uncased')
tokenizer = tokenization.FullTokenizer(vocab_file=VOCAB_FILE, do_lower_case=DO_LOWER_CASE)


In [0]:
def convert_to_unicode(text):
  """Converts `text` to Unicode (if it's not already), assuming utf-8 input."""
  if six.PY3:
    if isinstance(text, str):
      return text
    elif isinstance(text, bytes):
      return text.decode("utf-8", "ignore")
    else:
      raise ValueError("Unsupported string type: %s" % (type(text)))
  elif six.PY2:
    if isinstance(text, str):
      return text.decode("utf-8", "ignore")
    elif isinstance(text, unicode):
      return text
    else:
      raise ValueError("Unsupported string type: %s" % (type(text)))
  else:
    raise ValueError("Not running on Python2 or Python 3?")

In [0]:
import six
import collections
vocab = collections.OrderedDict()
index = 0
with tf.gfile.GFile(path, "r") as reader:
    while True:
      token = convert_to_unicode(reader.readline())
      if not token:
        break
      token = token.strip()
      vocab[token] = index
      index += 1

In [10]:
import pickle
tf.logging.set_verbosity(tf.logging.ERROR)
try:
  train_features = pickle.load(open('train_features_large_v2.dms','rb'))
except Exception as e: 
    print('can not load train features, creating train features: %s'%str(e))
#     train_features = pickle.load(tf.gfile.GFile('gs://bert_example/mask_lm/models/V2_augment/doc_leve/last_2/smallBERT-docLevel-seq512/data/train_features_large.dms', "rb"))


  
    train_features = run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
    with open('train_features_large_v2','wb') as f:
      pickle.dump(train_features,f)
try:

  dev_features = pickle.load(open('dev_features_large_ori.dms','rb'))
except Exception as e:

   
    print('can not load from GC, creating dev features')
    dev_features = run_classifier.convert_examples_to_features(dev_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
    with open('dev_features_large','wb') as f:
      pickle.dump(dev_features,f)
print(len(train_features))
print(len(dev_features))
# test_features = run_classifier.convert_examples_to_features(test_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
# test_fixed_features = run_classifier.convert_examples_to_features(test_InputExamples_fixed, label_list, MAX_SEQ_LENGTH, tokenizer)

can not load train features, creating train features: 
122150
3479


In [0]:
reset_selective -f '\btest\b'

In [69]:
# reset_selective -f data_train
# who_ls
import sys
ipython_vars =[]# ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']


sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)


[('test_fixed', 58314430),
 ('data_dev', 33318879),
 ('dev', 33263215),
 ('train_InputExamples', 4886104),
 ('train_features', 1043568),
 ('true_label', 549792),
 ('labels_val', 514576),
 ('lines', 253640),
 ('test_InputExamples_fixed', 201624),
 ('test_InputExamples', 161024),
 ('dev_InputExamples', 139264),
 ('train_label', 122254),
 ('dev_features', 30128),
 ('test_fixed_label', 5142),
 ('test_label', 4127),
 ('Out', 1048),
 ('auth_info', 1048),
 ('AdamWeightDecayOptimizer', 904),
 ('In', 648),
 ('f', 144),
 ('create_model', 120),
 ('create_optimizer', 120),
 ('load_dataset', 120),
 ('load_directory_data', 120),
 ('model_eval', 120),
 ('model_fn', 120),
 ('model_fn_builder', 120),
 ('model_predict', 120),
 ('model_train', 120),
 ('OUTPUT_DIR', 119),
 ('CONFIG_FILE', 109),
 ('INIT_CHECKPOINT', 108),
 ('VOCAB_FILE', 97),
 ('path', 97),
 ('BERT_MODEL_HUB', 92),
 ('BERT_PRETRAINED_DIR', 92),
 ('class_weights_arr', 88),
 ('label_list', 88),
 ('get_ipython', 80),
 ('absolute_import', 72),

In [0]:
# train_label = train['label']
test_label = test['label']
test_fixed_label = test_fixed['label']

In [68]:
import gc
gc.collect()

7

In [0]:
neg_w = 1./len(dev[dev['label']==0])
pos_w = 1./len(dev[dev['label']==1])
trainable = True
class_weights_arr = [neg_w/(neg_w+pos_w),pos_w/(neg_w+pos_w)]
def create_model(bert_config,is_training, input_ids, input_mask, segment_ids, labels,
                 num_labels,use_one_hot_embeddings):#, bert_hub_module_handle):
  """Creates a classification model."""
#    model = modeling.BertModel(
#       config=bert_config,
#       is_training=is_training,
#       input_ids=input_ids,
#       input_mask=input_mask,
#       token_type_ids=segment_ids,
#       use_one_hot_embeddings=use_one_hot_embeddings,)
    
  tags = set()
  if is_training:
    tags.add("train")
  bert_module = hub.Module(BERT_MODEL_HUB, tags=tags, trainable=trainable)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # In the demo, we are doing a simple classification task on the entire
  # segment.
  #
  # If you want to use the token-level output, use
  # bert_outputs["sequence_output"] instead.
  output_layer = bert_outputs["pooled_output"]

  hidden_size = output_layer.shape[-1].value

  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))
###### TO DO add weights
  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):
    if is_training:
      # I.e., 0.1 dropout
      output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    probabilities = tf.nn.softmax(logits, axis=-1)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)
    print(one_hot_labels.get_shape())
        
    one_hot_labels = one_hot_labels * class_weights_arr
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
       
    return (loss, per_example_loss, logits, probabilities)


def model_fn_builder(bert_config, num_labels,init_checkpoint, learning_rate, num_train_steps,
                     num_warmup_steps, use_tpu,use_one_hot_embeddings):# bert_hub_module_handle):
  """Returns `model_fn` closure for TPUEstimator."""

  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    tf.logging.info("*** Features ***")
    for name in sorted(features.keys()):
      tf.logging.info("  name = %s, shape = %s" % (name, features[name].shape))

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_training = (mode == tf.estimator.ModeKeys.TRAIN)

    (total_loss, per_example_loss, logits, probabilities) =create_model(
        bert_config, is_training, input_ids, input_mask, segment_ids, label_ids, num_labels,use_one_hot_embeddings)

    
    tvars = tf.trainable_variables()
    initialized_variable_names = {}
    scaffold_fn = None
    if init_checkpoint:
      (assignment_map, initialized_variable_names
      ) = modeling.get_assignment_map_from_checkpoint(tvars, init_checkpoint)
      if use_tpu:

        def tpu_scaffold():
          tf.train.init_from_checkpoint(init_checkpoint, assignment_map)
          return tf.train.Scaffold()

        scaffold_fn = tpu_scaffold
      else:
        tf.train.init_from_checkpoint(init_checkpoint, assignment_map)

    tf.logging.info("**** Trainable Variables ****")
    for var in tvars:
      init_string = ""
      if var.name in initialized_variable_names:
        init_string = ", *INIT_FROM_CKPT*"
      tf.logging.info("  name = %s, shape = %s%s", var.name, var.shape,
                      init_string)


    output_spec = None
    if mode == tf.estimator.ModeKeys.TRAIN:
     
      train_op = create_optimizer(
          total_loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu)

      output_spec = tf.contrib.tpu.TPUEstimatorSpec(
          mode=mode,
          loss=total_loss,
          train_op=train_op,scaffold_fn=scaffold_fn)
#       output_spec = tf.Print(output_spec,[tf_loss,total_loss],message='tfloss, total loss')
    elif mode == tf.estimator.ModeKeys.EVAL:

      def metric_fn(per_example_loss, label_ids, logits, is_real_example):
        predictions = tf.argmax(logits, axis=-1, output_type=tf.int32)
        accuracy = tf.metrics.accuracy(label_ids, predictions,weights=is_real_example)
        loss = tf.metrics.mean(per_example_loss,weights=is_real_example)
        return {
            "eval_accuracy": accuracy,
            "eval_loss": loss,
        }

      eval_metrics = (metric_fn, [per_example_loss, label_ids, logits])
      output_spec = tf.contrib.tpu.TPUEstimatorSpec(
          mode=mode,
          loss=total_loss,
          eval_metrics=eval_metrics,scaffold_fn=scaffold_fn)
    elif mode == tf.estimator.ModeKeys.PREDICT:
      output_spec = tf.contrib.tpu.TPUEstimatorSpec(
          mode=mode, predictions={"probabilities": probabilities},scaffold_fn=scaffold_fn)
    else:
      raise ValueError(
          "Only TRAIN, EVAL and PREDICT modes are supported: %s" % (mode))

    return output_spec

  return model_fn

In [0]:
# Force TF Hub writes to the GS bucket we provide.
os.environ['TFHUB_CACHE_DIR'] = OUTPUT_DIR
### Activate it if ** part is not activated 
model_fn = model_fn_builder(
    bert_config=modeling.BertConfig.from_json_file(CONFIG_FILE),
    num_labels=len(label_list),
    init_checkpoint=INIT_CHECKPOINT,
    learning_rate=LEARNING_RATE,
    num_train_steps=num_train_steps,
    num_warmup_steps=num_warmup_steps,
    use_tpu=True,
    use_one_hot_embeddings=True
#   bert_hub_module_handle=BERT_MODEL_HUB
)

# estimator = tf.contrib.tpu.TPUEstimator(
#   use_tpu=True,
#   model_fn=model_fn,
#   config=get_run_config(OUTPUT_DIR),
#   train_batch_size=TRAIN_BATCH_SIZE,
#   eval_batch_size=EVAL_BATCH_SIZE,
#   predict_batch_size=PREDICT_BATCH_SIZE, 
# )
# #####################################################################
## No Error
run_config = tf.contrib.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    model_dir=OUTPUT_DIR,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=ITERATIONS_PER_LOOP,
        num_shards=NUM_TPU_CORES,
        per_host_input_for_training=tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2))
# model_fn = run_classifier.model_fn_builder(
#     bert_config=modeling.BertConfig.from_json_file(CONFIG_FILE),
#     num_labels=len(label_list),
#     init_checkpoint=INIT_CHECKPOINT,
#     learning_rate=LEARNING_RATE,
#     num_train_steps=num_train_steps,
#     num_warmup_steps=num_warmup_steps,
#     use_tpu=use_tpu,
#     use_one_hot_embeddings=True)

estimator = tf.contrib.tpu.TPUEstimator(
    use_tpu=use_tpu,
    model_fn=model_fn,
    config=run_config,
    train_batch_size=TRAIN_BATCH_SIZE,
    eval_batch_size=EVAL_BATCH_SIZE,
    predict_batch_size=PREDICT_BATCH_SIZE)

# estimator_from_tfhub._export_to_tpu = False


In [0]:
# Train the model
# tf.logging.set_verbosity(tf.logging.FATAL) #DEBUG,ERROR,FATAL,INFO,WARN
def model_train(estimator,train_features=train_features):
  # We'll set sequences to be at most 128 tokens long.

  print('***** Started training at {} *****'.format(datetime.datetime.now()))
  print('  Num examples = {}'.format(len(train_features)))
  print('  Batch size = {}'.format(TRAIN_BATCH_SIZE))
  tf.logging.info("  Num steps = %d", num_train_steps)
  train_input_fn = run_classifier.input_fn_builder(
      features=train_features,
      seq_length=MAX_SEQ_LENGTH,
      is_training=True,
      drop_remainder=True)
  print('start running estimator')
#   estimator._export_to_tpu = False
  md = estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
  print('***** Finished training at {} *****'.format(datetime.datetime.now()))
  return md



#Evaluation and Prediction 

In [0]:
def model_eval(estimator,eval_examples=dev_InputExamples,eval_features=dev_features):
  # Eval the model.
#   eval_examples = dev_InputExamples#processor.get_dev_examples(TASK_DATA_DIR)
#   eval_features = run_classifier.convert_examples_to_features(
#       eval_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  print('***** Started evaluation at {} *****'.format(datetime.datetime.now()))
  print('  Num examples = {}'.format(len(eval_examples)))
  print('  Batch size = {}'.format(EVAL_BATCH_SIZE))

  # Eval will be slightly WRONG on the TPU because it will truncate
  # the last batch.
  eval_steps = int(len(eval_examples) / EVAL_BATCH_SIZE)
  eval_input_fn = run_classifier.input_fn_builder(
      features=eval_features,
      seq_length=MAX_SEQ_LENGTH,
      is_training=False,
      drop_remainder=True)
  result = estimator.evaluate(input_fn=eval_input_fn, steps=eval_steps)
  print('***** Finished evaluation at {} *****'.format(datetime.datetime.now()))
  output_eval_file = os.path.join(OUTPUT_DIR, "eval_results.txt")
  with tf.gfile.GFile(output_eval_file, "w") as writer:
    print("***** Eval results *****")
    for key in sorted(result.keys()):
      print('  {} = {}'.format(key, str(result[key])))
      writer.write("%s = %s\n" % (key, str(result[key])))
      
  return result

In [0]:
import numpy as np
from sklearn import metrics
# labels = [1,0]
# def model_predict(estimator,prediction_examples):
#   # Make predictions on a subset of eval examples
# #   prediction_examples = processor.get_dev_examples(TASK_DATA_DIR)[:PREDICT_BATCH_SIZE]
#   input_features = run_classifier.convert_examples_to_features(prediction_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
#   predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=True)
#   predictions = estimator.predict(predict_input_fn)
#   return [(sentence, prediction['probabilities']) for sentence, prediction in zip(prediction_examples, predictions)]

def model_predict(estimator,input_features,input_examples,checkpoint_path=None):
  # Make predictions on a subset of eval examples
#   prediction_examples = processor.get_dev_examples(TASK_DATA_DIR)[:PREDICT_BATCH_SIZE]
#   input_features = run_classifier.convert_examples_to_features(input_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=True)
#   predictions = estimator.predict(predict_input_fn)
  if checkpoint_path: 
    predictions = estimator.predict(predict_input_fn,checkpoint_path=checkpoint_path)
  else:
    predictions = estimator.predict(predict_input_fn)


  return [(sentence, prediction['probabilities']) for sentence, prediction in zip(input_examples, predictions)]



In [17]:
pd = model_predict(estimator,dev_features[:3100],dev_InputExamples[:3100])
true_label = list(dev['label'][:3100])
labels_val = []
for item in pd:
    labels_val.append(np.argmax(item[1]))
print(metrics.confusion_matrix(y_pred=labels_val,y_true=true_label))
print(metrics.classification_report(y_pred=labels_val,y_true = true_label))

Instructions for updating:
Colocations handled automatically by placer.


W0515 23:39:17.925529 140630432184192 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow/python/ops/resource_variable_ops.py:435: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


INFO:tensorflow:Querying Tensorflow master (grpc://10.77.130.114:8470) for TPU system metadata.


I0515 23:39:17.933187 140630432184192 tpu_system_metadata.py:59] Querying Tensorflow master (grpc://10.77.130.114:8470) for TPU system metadata.


INFO:tensorflow:Found TPU system:


...

INFO:tensorflow:Dequeue next (1) batch(es) of data from outfeed.


I0515 23:40:19.836847 140630432184192 tpu_estimator.py:540] Dequeue next (1) batch(es) of data from outfeed.


INFO:tensorflow:prediction_loop marked as finished


I0515 23:40:20.029902 140630432184192 error_handling.py:93] prediction_loop marked as finished


[[ 260  841]
 [ 478 1521]]
              precision    recall  f1-score   support

       False       0.35      0.24      0.28      1101
        True       0.64      0.76      0.70      1999

   micro avg       0.57      0.57      0.57      3100
   macro avg       0.50      0.50      0.49      3100
weighted avg       0.54      0.57      0.55      3100



In [18]:
tf.logging.set_verbosity(tf.logging.FATAL)
test_features = run_classifier.convert_examples_to_features(test_InputExamples_fixed, label_list, MAX_SEQ_LENGTH, tokenizer)
pd = model_predict(estimator,test_features,test_InputExamples_fixed)
true_label = list(test_fixed['label'])
labels_val = []
for item in pd:
    labels_val.append(np.argmax(item[1]))
print(metrics.confusion_matrix(y_pred=labels_val,y_true=true_label))
print(metrics.classification_report(y_pred=labels_val,y_true = true_label))

(1, 2)
[[ 738 1080]
 [ 241 2979]]
              precision    recall  f1-score   support

       False       0.75      0.41      0.53      1818
        True       0.73      0.93      0.82      3220

   micro avg       0.74      0.74      0.74      5038
   macro avg       0.74      0.67      0.67      5038
weighted avg       0.74      0.74      0.71      5038



In [30]:
tf.logging.set_verbosity(tf.logging.FATAL)
# model_train(estimator_from_tfhub)
model_train(estimator,train_features=train_features[:61075])

***** Started training at 2019-05-17 04:34:12.359480 *****
  Num examples = 61075
  Batch size = 8
start running estimator
(1, 2)
excluded trainable variables: >> [<tf.Variable 'module/bert/embeddings/word_embeddings:0' shape=(30522, 768) dtype=float32>, <tf.Variable 'module/bert/embeddings/token_type_embeddings:0' shape=(2, 768) dtype=float32>, <tf.Variable 'module/bert/embeddings/position_embeddings:0' shape=(512, 768) dtype=float32>, <tf.Variable 'module/bert/embeddings/LayerNorm/beta:0' shape=(768,) dtype=float32>, <tf.Variable 'module/bert/embeddings/LayerNorm/gamma:0' shape=(768,) dtype=float32>, <tf.Variable 'module/bert/encoder/layer_10/attention/self/query/kernel:0' shape=(768, 768) dtype=float32>, <tf.Variable 'module/bert/encoder/layer_10/attention/self/query/bias:0' shape=(768,) dtype=float32>, <tf.Variable 'module/bert/encoder/layer_10/attention/self/key/kernel:0' shape=(768, 768) dtype=float32>, <tf.Variable 'module/bert/encoder/layer_10/attention/self/key/bias:0' shape=(

/usr/local/lib/python2.7/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


***** Finished training at 2019-05-17 08:28:49.097394 *****


In [74]:
# pd = model_predict(estimator,train_features[:61075],train_InputExamples[:61075],checkpoint_path=OUTPUT_DIR+'/model.ckpt-118000')
# # true_label = list(dev['label'][:])
# true_label = list(train_label[:61075])
# labels_val = []
# for item in pd:
#     labels_val.append(np.argmax(item[1]))
# print(metrics.confusion_matrix(y_pred=labels_val,y_true=true_label))
# print(metrics.classification_report(y_pred=labels_val,y_true = true_label))

%xdel 

NameError: name 'train' is not defined


In [0]:
del pd

In [0]:
## Run it if you want to train for a range of epochs and see the validation error and save the prediction on than
## ** Part Name **
# mds = []
# evs = []
pds_dev = []
pds_tr = []
tf.logging.set_verbosity(tf.logging.FATAL) 

for i in range(1,11):
  print('----------------------- Starting Epoch %d-----------------------'%i)

  NUM_TRAIN_EPOCHS = i
  
  num_train_steps = int(len(train_InputExamples) / TRAIN_BATCH_SIZE * NUM_TRAIN_EPOCHS)
  num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

#   model_fn = model_fn_builder(
#   num_labels=len(label_list),
#   learning_rate=LEARNING_RATE,
#   num_train_steps=num_train_steps,
#   num_warmup_steps=num_warmup_steps,
#   use_tpu=True,
#   bert_hub_module_handle=BERT_MODEL_HUB)

  model_fn = model_fn_builder(
  bert_config=modeling.BertConfig.from_json_file(CONFIG_FILE),
  num_labels=len(label_list),
  init_checkpoint=INIT_CHECKPOINT,
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps,
  use_tpu=True,
  use_one_hot_embeddings=True)
  
  
  estimator_from_tfhub = tf.contrib.tpu.TPUEstimator(
  use_tpu=True,
  model_fn=model_fn,
  config=run_config,
  train_batch_size=TRAIN_BATCH_SIZE,
  eval_batch_size=EVAL_BATCH_SIZE,
  predict_batch_size=PREDICT_BATCH_SIZE)
  
  model_train(estimator_from_tfhub)
#   ev = model_eval(estimator_from_tfhub)

#   print(' -------------------- Train Prediction --------------------')
#   pd = model_predict(estimator_from_tfhub,train_features,train_InputExamples)
#   true_label = list(train['label'])
#   pds_tr.append(pd)
#   labels_val = []
#   for item in pd:
#     labels_val.append(np.argmax(item[1]))
#   print(metrics.confusion_matrix(y_pred=labels_val,y_true=true_label))
#   print(metrics.classification_report(y_pred=labels_val,y_true = true_label))
  
  
  print(' -------------------- Dev Prediction --------------------')
  pd = model_predict(estimator_from_tfhub,dev_features[:3100],dev_InputExamples[:3100])
  true_label = list(dev['label'][:3100])
  pds_dev.append(pd)
  labels_val = []
  for item in pd:
    labels_val.append(np.argmax(item[1]))
  print(metrics.confusion_matrix(y_pred=labels_val,y_true=true_label))
  print(metrics.classification_report(y_pred=labels_val,y_true = true_label))




for tr,dv in zip(pds_tr,pds_dev):
  labels_val = []
  true_label = list(train['label'])
  for item in tr:
    labels_val.append(np.argmax(item[1]))
  print(metrics.confusion_matrix(y_pred=labels_val,y_true=true_label))
  print(metrics.classification_report(y_pred=labels_val,y_true = true_label))
  
  labels_val = []
  true_label = list(dev['label'][:3100])
  for item in dv:
    labels_val.append(np.argmax(item[1]))
  print(metrics.confusion_matrix(y_pred=labels_val,y_true=true_label))
  print(metrics.classification_report(y_pred=labels_val,y_true = true_label))


----------------------- Starting Epoch 1-----------------------
***** Started training at 2019-05-15 17:57:14.080193 *****
  Num examples = 122150
  Batch size = 8
start running estimator
(1, 2)
excluded trainable variables: >> [<tf.Variable 'module/bert/embeddings/word_embeddings:0' shape=(30522, 768) dtype=float32>, <tf.Variable 'module/bert/embeddings/token_type_embeddings:0' shape=(2, 768) dtype=float32>, <tf.Variable 'module/bert/embeddings/position_embeddings:0' shape=(512, 768) dtype=float32>, <tf.Variable 'module/bert/embeddings/LayerNorm/beta:0' shape=(768,) dtype=float32>, <tf.Variable 'module/bert/embeddings/LayerNorm/gamma:0' shape=(768,) dtype=float32>, <tf.Variable 'module/bert/encoder/layer_10/attention/self/query/kernel:0' shape=(768, 768) dtype=float32>, <tf.Variable 'module/bert/encoder/layer_10/attention/self/query/bias:0' shape=(768,) dtype=float32>, <tf.Variable 'module/bert/encoder/layer_10/attention/self/key/kernel:0' shape=(768, 768) dtype=float32>, <tf.Variable

/usr/local/lib/python2.7/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [0]:
model_fn = model_fn_builder(
  bert_config=modeling.BertConfig.from_json_file(CONFIG_FILE),
  num_labels=len(label_list),
  init_checkpoint=INIT_CHECKPOINT,
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps,
  use_tpu=True,
  use_one_hot_embeddings=True)
  
estimator_from_tfhub = tf.contrib.tpu.TPUEstimator(
  use_tpu=True,
  model_fn=model_fn,
  config=run_config,
  train_batch_size=TRAIN_BATCH_SIZE,
  eval_batch_size=EVAL_BATCH_SIZE,
  predict_batch_size=PREDICT_BATCH_SIZE)
  
pd = model_predict(estimator_from_tfhub,dev_features[:3000],dev_InputExamples[:3000])
true_label = list(dev['label'][:3000])
labels_val = []
for item in pd:
    labels_val.append(np.argmax(item[1]))
print(metrics.confusion_matrix(y_pred=labels_val,y_true=true_label))
print(metrics.classification_report(y_pred=labels_val,y_true = true_label))


(4, 2)
[[ 684  386]
 [ 123 1807]]
              precision    recall  f1-score   support

       False       0.85      0.64      0.73      1070
        True       0.82      0.94      0.88      1930

   micro avg       0.83      0.83      0.83      3000
   macro avg       0.84      0.79      0.80      3000
weighted avg       0.83      0.83      0.82      3000



In [0]:
# prd = model_predict(estimator,train_features,train_InputExamples)
# prd = [item for item in predictions]
len(prd)

(4, 2)


InvalidArgumentError: ignored

In [0]:
len(a)
# import numpy as np
# from sklearn import metrics

# labels = ["Negative","Neutral", "Positive"]
# labels_val = []
# for item in predictions:
#   labels_val.append(labels[np.argmax(item[1])])
# true_label = list(dev['sentiment'])
# print(metrics.confusion_matrix(y_pred=labels_val,y_true=true_label))
# print(metrics.classification_report(y_pred=labels_val,y_true = true_label))
# labels_val = []
# for item in predictions:
#   labels_val.append(np.argmax(item[1]))
# true_label = list(train['label'])
# print(metrics.confusion_matrix(y_pred=labels_val,y_true=true_label))
# print(metrics.classification_report(y_pred=labels_val,y_true = true_label))

[[  0  48]
 [  0 152]]
              precision    recall  f1-score   support

       False       0.00      0.00      0.00        48
        True       0.76      1.00      0.86       152

   micro avg       0.76      0.76      0.76       200
   macro avg       0.38      0.50      0.43       200
weighted avg       0.58      0.76      0.66       200



In [0]:
predictions = model_predict(estimator_from_tfhub,test_InputExamples_fixed)
labels_val = []
for item in predictions:
  labels_val.append(labels[np.argmax(item[1])])
true_label = list(test_fixed['sentiment'])




NameError: ignored

In [0]:
print(metrics.confusion_matrix(y_pred=labels_val,y_true=true_label))
print(metrics.classification_report(y_pred=labels_val,y_true = true_label))

In [0]:
# tf.logging.set_verbosity(tf.logging.DEBUG) #DEBUG,ERROR,FATAL,INFO,WARN
# predictions = model_predict(estimator_from_tfhub,test_InputExamples)
labels_val = []
for item in predictions:
  labels_val.append(labels[np.argmax(item[1])])
true_label = list(test['sentiment'])


In [0]:
# len(labels_val)
# len(test_InputExamples)
# len(predictions)
model_eval(estimator_from_tfhub)

***** Started evaluation at 2019-05-09 17:10:37.771525 *****
  Num examples = 3479
  Batch size = 8
(1, 2)


RuntimeError: ignored

In [0]:
print(metrics.confusion_matrix(y_pred=labels_val,y_true=true_label))
print(metrics.classification_report(y_pred=labels_val,y_true = true_label))

In [0]:
####seq 128, Small BERT, Batchsize 32 for train, 8 for dev and test
# 1)I0423 16:57:01.507206 140076901431168 basic_session_run_hooks.py:249] loss = 0.40919852, step = 46
# 2)I0423 18:08:31.116359 140500355729280 basic_session_run_hooks.py:249] loss = 0.7756149, step = 92
# 4)Loss for final step: 0.6891643.
# 5)Loss for final step: 0.9730371
# 6)Loss for final step: 0.4775733
# 7)Loss for final step: 1.2802429.
# 8)Loss for final step: 0.509207
# 9)loss = 0.29262337, step = 414
# 10)Loss for final step: 0.47267017


# 2)***** Eval results *****
#   eval_accuracy = 0.5857143
#   eval_loss = 0.8630275
#   global_step = 92
#   loss = 0.79767215
# 3)***** Eval results *****
#   eval_accuracy = 0.5857143
#   eval_loss = 0.8630275
#   global_step = 138
#   loss = 0.79767215
# 4)***** Eval results *****
#   eval_accuracy = 0.5857143
#   eval_loss = 0.86550355
#   global_step = 184
#   loss = 0.83540887
# 5)***** Eval results *****
#   eval_accuracy = 0.5964286
#   eval_loss = 0.88053304
#   global_step = 230
#   loss = 0.91362196
# 6)***** Eval results *****
#   eval_accuracy = 0.5857143
#   eval_loss = 0.90498805
#   global_step = 276
#   loss = 1.0806552
# 7)***** Eval results *****
#   eval_accuracy = 0.56785715
#   eval_loss = 0.92742974
#   global_step = 322
#   loss = 1.0180835
# 8)***** Eval results *****
#   eval_accuracy = 0.5607143
#   eval_loss = 0.9436406
#   global_step = 368
#   loss = 0.9721719
# 9)***** Eval results *****
#   eval_accuracy = 0.5607143
#   eval_loss = 0.9714315
#   global_step = 414
#   loss = 0.5798999
# 10)***** Eval results *****
#   eval_accuracy = 0.54285717
#   eval_loss = 1.0072392
#   global_step = 460
#   loss = 1.053762
  
#   DEV Info:
# 1)[[  0   0  23]
#  [  0   0  97]
#  [  0   0 164]]
#               precision    recall  f1-score   support

#     Negative       0.00      0.00      0.00        23
#      Neutral       0.00      0.00      0.00        97
#     Positive       0.58      1.00      0.73       164

#    micro avg       0.58      0.58      0.58       284
#    macro avg       0.19      0.33      0.24       284
# weighted avg       0.33      0.58      0.42       284
# 2)[[  0   9  14]
#  [  0  25  72]
#  [  0  21 143]]
#               precision    recall  f1-score   support

#     Negative       0.00      0.00      0.00        23
#      Neutral       0.45      0.26      0.33        97
#     Positive       0.62      0.87      0.73       164

#    micro avg       0.59      0.59      0.59       284
#    macro avg       0.36      0.38      0.35       284
# weighted avg       0.52      0.59      0.53       284
# 3)[[  0   9  14]
#  [  0  25  72]
#  [  0  21 143]]
#               precision    recall  f1-score   support

#     Negative       0.00      0.00      0.00        23
#      Neutral       0.45      0.26      0.33        97
#     Positive       0.62      0.87      0.73       164

#    micro avg       0.59      0.59      0.59       284
#    macro avg       0.36      0.38      0.35       284
# weighted avg       0.52      0.59      0.53       284
# 4)[[  0   8  42]
#  [  0  16 100]
#  [  0  17 157]]
#               precision    recall  f1-score   support

#     Negative       0.00      0.00      0.00        50
#      Neutral       0.39      0.14      0.20       116
#     Positive       0.53      0.90      0.66       174

#    micro avg       0.51      0.51      0.51       340
#    macro avg       0.31      0.35      0.29       340
# weighted avg       0.40      0.51      0.41       340
# 5)[[  0  17   6]
#  [  0  40  57]
#  [  0  35 129]]
#               precision    recall  f1-score   support

#     Negative       0.00      0.00      0.00        23
#      Neutral       0.43      0.41      0.42        97
#     Positive       0.67      0.79      0.72       164

#    micro avg       0.60      0.60      0.60       284
#    macro avg       0.37      0.40      0.38       284
# weighted avg       0.54      0.60      0.56       284
# 6)[[  0  19   4]
#  [  0  44  53]
#  [  0  42 122]]
#               precision    recall  f1-score   support

#     Negative       0.00      0.00      0.00        23
#      Neutral       0.42      0.45      0.44        97
#     Positive       0.68      0.74      0.71       164

#    micro avg       0.58      0.58      0.58       284
#    macro avg       0.37      0.40      0.38       284
# weighted avg       0.54      0.58      0.56       284
# 7)[[  0  18   5]
#  [  0  46  51]
#  [  0  47 117]]
#               precision    recall  f1-score   support

#     Negative       0.00      0.00      0.00        23
#      Neutral       0.41      0.47      0.44        97
#     Positive       0.68      0.71      0.69       164

#    micro avg       0.57      0.57      0.57       284
#    macro avg       0.36      0.40      0.38       284
# weighted avg       0.53      0.57      0.55       284
# 8)[[  0  18   5]
#  [  0  48  49]
#  [  0  53 111]]
#               precision    recall  f1-score   support

#     Negative       0.00      0.00      0.00        23
#      Neutral       0.40      0.49      0.44        97
#     Positive       0.67      0.68      0.67       164

#    micro avg       0.56      0.56      0.56       284
#    macro avg       0.36      0.39      0.37       284
# weighted avg       0.53      0.56      0.54       284
# 9)[[  0  18   5]
#  [  0  48  49]
#  [  0  54 110]]
#               precision    recall  f1-score   support

#     Negative       0.00      0.00      0.00        23
#      Neutral       0.40      0.49      0.44        97
#     Positive       0.67      0.67      0.67       164

#    micro avg       0.56      0.56      0.56       284
#    macro avg       0.36      0.39      0.37       284
# weighted avg       0.52      0.56      0.54       284
# 10)[[  0  18   5]
#  [  0  51  46]
#  [  0  62 102]]
#               precision    recall  f1-score   support

#     Negative       0.00      0.00      0.00        23
#      Neutral       0.39      0.53      0.45        97
#     Positive       0.67      0.62      0.64       164

#    micro avg       0.54      0.54      0.54       284
#    macro avg       0.35      0.38      0.36       284
# weighted avg       0.52      0.54      0.52       284


In [0]:
###### Seq:256, small BERT, batch size 32 for train, 8 for test and dev
# 3) {'loss': 0.79650426, 'eval_accuracy': 0.58214283, 'eval_loss': 0.86125755, 'global_step': 138}
# [[  0   0  23]
#  [  0   0  97]
#  [  0   0 164]]
#               precision    recall  f1-score   support

#     Negative       0.00      0.00      0.00        23
#      Neutral       0.00      0.00      0.00        97
#     Positive       0.58      1.00      0.73       164

#    micro avg       0.58      0.58      0.58       284
#    macro avg       0.19      0.33      0.24       284
# weighted avg       0.33      0.58      0.42       284

# 4) {'loss': 0.73700047, 'eval_accuracy': 0.5928571, 'eval_loss': 0.8223035, 'global_step': 184}
# [[  0  11  12]
#  [  0  20  77]
#  [  0  17 147]]
#               precision    recall  f1-score   support

#     Negative       0.00      0.00      0.00        23
#      Neutral       0.42      0.21      0.28        97
#     Positive       0.62      0.90      0.73       164

#    micro avg       0.59      0.59      0.59       284
#    macro avg       0.35      0.37      0.34       284
# weighted avg       0.50      0.59      0.52       284

# 5) {'loss': 0.71594626, 'eval_accuracy': 0.5928571, 'eval_loss': 0.82239425, 'global_step': 230}
# [[  0  19   4]
#  [  0  51  46]
#  [  0  48 116]]
#               precision    recall  f1-score   support

#     Negative       0.00      0.00      0.00        23
#      Neutral       0.43      0.53      0.47        97
#     Positive       0.70      0.71      0.70       164

#    micro avg       0.59      0.59      0.59       284
#    macro avg       0.38      0.41      0.39       284
# weighted avg       0.55      0.59      0.57       284

# 6) {'loss': 0.7227276, 'eval_accuracy': 0.5857143, 'eval_loss': 0.8479867, 'global_step': 276}
# [[  0  21   2]
#  [  2  51  44]
#  [  1  49 114]]
#               precision    recall  f1-score   support

#     Negative       0.00      0.00      0.00        23
#      Neutral       0.42      0.53      0.47        97
#     Positive       0.71      0.70      0.70       164

#    micro avg       0.58      0.58      0.58       284
#    macro avg       0.38      0.41      0.39       284
# weighted avg       0.56      0.58      0.57       284

# 7){'loss': 0.83304703, 'eval_accuracy': 0.5642857, 'eval_loss': 0.90589315, 'global_step': 322}
# [[  0  19   4]
#  [  4  40  53]
#  [  3  42 119]]
#               precision    recall  f1-score   support

#     Negative       0.00      0.00      0.00        23
#      Neutral       0.40      0.41      0.40        97
#     Positive       0.68      0.73      0.70       164

#    micro avg       0.56      0.56      0.56       284
#    macro avg       0.36      0.38      0.37       284
# weighted avg       0.53      0.56      0.54       284

# 8){'loss': 0.84049994, 'eval_accuracy': 0.575, 'eval_loss': 0.93640614, 'global_step': 368}
# [[  2  18   3]
#  [  5  42  50]
#  [  2  44 118]]
#               precision    recall  f1-score   support

#     Negative       0.22      0.09      0.12        23
#      Neutral       0.40      0.43      0.42        97
#     Positive       0.69      0.72      0.70       164

#    micro avg       0.57      0.57      0.57       284
#    macro avg       0.44      0.41      0.42       284
# weighted avg       0.55      0.57      0.56       284

# 9){'loss': 0.8601472, 'eval_accuracy': 0.5642857, 'eval_loss': 0.95250976, 'global_step': 414}
# [[  0  18   5]
#  [  4  37  56]
#  [  1  41 122]]
#               precision    recall  f1-score   support

#     Negative       0.00      0.00      0.00        23
#      Neutral       0.39      0.38      0.38        97
#     Positive       0.67      0.74      0.70       164

#    micro avg       0.56      0.56      0.56       284
#    macro avg       0.35      0.38      0.36       284
# weighted avg       0.52      0.56      0.54       284


# 10){'loss': 0.9091368, 'eval_accuracy': 0.5535714, 'eval_loss': 0.9715489, 'global_step': 460}
# [[  0  19   4]
#  [  4  37  56]
#  [  1  44 119]]
#               precision    recall  f1-score   support

#     Negative       0.00      0.00      0.00        23
#      Neutral       0.37      0.38      0.38        97
#     Positive       0.66      0.73      0.69       164

#    micro avg       0.55      0.55      0.55       284
#    macro avg       0.34      0.37      0.36       284
# weighted avg       0.51      0.55      0.53       284

# 11){'loss': 0.9756337, 'eval_accuracy': 0.5714286, 'eval_loss': 1.0129306, 'global_step': 506}
# [[  3  16   4]
#  [  5  36  56]
#  [  1  41 122]]
#               precision    recall  f1-score   support

#     Negative       0.33      0.13      0.19        23
#      Neutral       0.39      0.37      0.38        97
#     Positive       0.67      0.74      0.71       164

#    micro avg       0.57      0.57      0.57       284
#    macro avg       0.46      0.42      0.42       284
# weighted avg       0.55      0.57      0.55       284

# 12){'loss': 0.9551747, 'eval_accuracy': 0.56785715, 'eval_loss': 1.0222387, 'global_step': 552}
# [[  3  17   3]
#  [  5  38  54]
#  [  2  43 119]]
#               precision    recall  f1-score   support

#     Negative       0.30      0.13      0.18        23
#      Neutral       0.39      0.39      0.39        97
#     Positive       0.68      0.73      0.70       164

#    micro avg       0.56      0.56      0.56       284
#    macro avg       0.45      0.42      0.42       284
# weighted avg       0.55      0.56      0.55       284

# 13){'loss': 1.0276382, 'eval_accuracy': 0.5714286, 'eval_loss': 1.0547612, 'global_step': 598}
# [[  3  17   3]
#  [  5  36  56]
#  [  1  41 122]]
#               precision    recall  f1-score   support

#     Negative       0.33      0.13      0.19        23
#      Neutral       0.38      0.37      0.38        97
#     Positive       0.67      0.74      0.71       164

#    micro avg       0.57      0.57      0.57       284
#    macro avg       0.46      0.42      0.42       284
# weighted avg       0.55      0.57      0.55       284

# 14){'loss': 1.0036505, 'eval_accuracy': 0.55714285, 'eval_loss': 1.0697843, 'global_step': 644}
# [[  3  17   3]
#  [  7  36  54]
#  [  2  44 118]]
#               precision    recall  f1-score   support

#     Negative       0.25      0.13      0.17        23
#      Neutral       0.37      0.37      0.37        97
#     Positive       0.67      0.72      0.70       164

#    micro avg       0.55      0.55      0.55       284
#    macro avg       0.43      0.41      0.41       284
# weighted avg       0.54      0.55      0.54       284

# 15){'loss': 1.0295725, 'eval_accuracy': 0.5642857, 'eval_loss': 1.0809377, 'global_step': 690}
# [[  3  17   3]
#  [  5  39  53]
#  [  1  46 117]]
#               precision    recall  f1-score   support

#     Negative       0.33      0.13      0.19        23
#      Neutral       0.38      0.40      0.39        97
#     Positive       0.68      0.71      0.69       164

#    micro avg       0.56      0.56      0.56       284
#    macro avg       0.46      0.42      0.42       284
# weighted avg       0.55      0.56      0.55       284

# 16){'loss': 1.0889318, 'eval_accuracy': 0.54642856, 'eval_loss': 1.1056138, 'global_step': 736}
# [[  3  17   3]
#  [  8  33  56]
#  [  2  44 118]]
#               precision    recall  f1-score   support

#     Negative       0.23      0.13      0.17        23
#      Neutral       0.35      0.34      0.35        97
#     Positive       0.67      0.72      0.69       164

#    micro avg       0.54      0.54      0.54       284
#    macro avg       0.42      0.40      0.40       284
# weighted avg       0.52      0.54      0.53       284

# 17){'loss': 1.1312778, 'eval_accuracy': 0.54642856, 'eval_loss': 1.1440269, 'global_step': 782}
# [[  3  17   3]
#  [  9  33  55]
#  [  2  44 118]]
#               precision    recall  f1-score   support

#     Negative       0.21      0.13      0.16        23
#      Neutral       0.35      0.34      0.35        97
#     Positive       0.67      0.72      0.69       164

#    micro avg       0.54      0.54      0.54       284
#    macro avg       0.41      0.40      0.40       284
# weighted avg       0.52      0.54      0.53       284

# 18){'loss': 1.1436831, 'eval_accuracy': 0.55, 'eval_loss': 1.1613237, 'global_step': 828}
# [[  3  16   4]
#  [  8  35  54]
#  [  2  45 117]]
#               precision    recall  f1-score   support

#     Negative       0.23      0.13      0.17        23
#      Neutral       0.36      0.36      0.36        97
#     Positive       0.67      0.71      0.69       164

#    micro avg       0.55      0.55      0.55       284
#    macro avg       0.42      0.40      0.41       284
# weighted avg       0.53      0.55      0.54       284

# 19){'loss': 1.1817628, 'eval_accuracy': 0.55, 'eval_loss': 1.1834545, 'global_step': 874}
# [[  3  16   4]
#  [  8  34  55]
#  [  1  45 118]]
#               precision    recall  f1-score   support

#     Negative       0.25      0.13      0.17        23
#      Neutral       0.36      0.35      0.35        97
#     Positive       0.67      0.72      0.69       164

#    micro avg       0.55      0.55      0.55       284
#    macro avg       0.42      0.40      0.41       284
# weighted avg       0.53      0.55      0.53       284

# 20){'loss': 1.1921012, 'eval_accuracy': 0.54642856, 'eval_loss': 1.1976833, 'global_step': 920}
# [[  3  16   4]
#  [  9  35  53]
#  [  2  47 115]]
#               precision    recall  f1-score   support

#     Negative       0.21      0.13      0.16        23
#      Neutral       0.36      0.36      0.36        97
#     Positive       0.67      0.70      0.68       164

#    micro avg       0.54      0.54      0.54       284
#    macro avg       0.41      0.40      0.40       284
# weighted avg       0.53      0.54      0.53       284

# 21){'loss': 1.2514663, 'eval_accuracy': 0.53571427, 'eval_loss': 1.2244278, 'global_step': 966}
# [[  3  16   4]
#  [ 11  32  54]
#  [  2  47 115]]
#               precision    recall  f1-score   support

#     Negative       0.19      0.13      0.15        23
#      Neutral       0.34      0.33      0.33        97
#     Positive       0.66      0.70      0.68       164

#    micro avg       0.53      0.53      0.53       284
#    macro avg       0.40      0.39      0.39       284
# weighted avg       0.51      0.53      0.52       284

# 22){'loss': 1.2630298, 'eval_accuracy': 0.5321429, 'eval_loss': 1.2527977, 'global_step': 1012}
# [[  5  14   4]
#  [ 14  32  51]
#  [  9  43 112]]
#               precision    recall  f1-score   support

#     Negative       0.18      0.22      0.20        23
#      Neutral       0.36      0.33      0.34        97
#     Positive       0.67      0.68      0.68       164

#    micro avg       0.52      0.52      0.52       284
#    macro avg       0.40      0.41      0.41       284
# weighted avg       0.52      0.52      0.52       284

# 23){'loss': 1.3330169, 'eval_accuracy': 0.53571427, 'eval_loss': 1.2815608, 'global_step': 1058}
# [[  6  13   4]
#  [ 14  31  52]
#  [  9  42 113]]
#               precision    recall  f1-score   support

#     Negative       0.21      0.26      0.23        23
#      Neutral       0.36      0.32      0.34        97
#     Positive       0.67      0.69      0.68       164

#    micro avg       0.53      0.53      0.53       284
#    macro avg       0.41      0.42      0.42       284
# weighted avg       0.53      0.53      0.53       284

# 24){'loss': 1.3111317, 'eval_accuracy': 0.5321429, 'eval_loss': 1.2898273, 'global_step': 1104}
# [[  5  14   4]
#  [ 14  32  51]
#  [  9  43 112]]
#               precision    recall  f1-score   support

#     Negative       0.18      0.22      0.20        23
#      Neutral       0.36      0.33      0.34        97
#     Positive       0.67      0.68      0.68       164

#    micro avg       0.52      0.52      0.52       284
#    macro avg       0.40      0.41      0.41       284
# weighted avg       0.52      0.52      0.52       284

In [0]:
## Larg BERT seq:256, batchsize 8 for train, dev , test
# {'loss': 1.0679293, 'eval_accuracy': 0.5955882, 'eval_loss': 0.8965841, 'global_step': 93}
# [[  0   0  23]
#  [  0   0  97]
#  [  0   0 164]]
#               precision    recall  f1-score   support

#     Negative       0.00      0.00      0.00        23
#      Neutral       0.00      0.00      0.00        97
#     Positive       0.58      1.00      0.73       164

#    micro avg       0.58      0.58      0.58       284
#    macro avg       0.19      0.33      0.24       284
# weighted avg       0.33      0.58      0.42       284

# {'loss': 1.0362501, 'eval_accuracy': 0.5955882, 'eval_loss': 0.88540924, 'global_step': 186}
# [[  0   0  23]
#  [  0   0  97]
#  [  0   0 164]]
#               precision    recall  f1-score   support

#     Negative       0.00      0.00      0.00        23
#      Neutral       0.00      0.00      0.00        97
#     Positive       0.58      1.00      0.73       164

#    micro avg       0.58      0.58      0.58       284
#    macro avg       0.19      0.33      0.24       284
# weighted avg       0.33      0.58      0.42       284

# {'loss': 1.0290864, 'eval_accuracy': 0.5955882, 'eval_loss': 0.8812368, 'global_step': 279}
# [[  0   0  23]
#  [  0   0  97]
#  [  0   0 164]]
#               precision    recall  f1-score   support

#     Negative       0.00      0.00      0.00        23
#      Neutral       0.00      0.00      0.00        97
#     Positive       0.58      1.00      0.73       164

#    micro avg       0.58      0.58      0.58       284
#    macro avg       0.19      0.33      0.24       284
# weighted avg       0.33      0.58      0.42       284

# {'loss': 1.0271251, 'eval_accuracy': 0.5955882, 'eval_loss': 0.88323754, 'global_step': 372}
# [[  0   0  23]
#  [  0   0  97]
#  [  0   0 164]]
#               precision    recall  f1-score   support

#     Negative       0.00      0.00      0.00        23
#      Neutral       0.00      0.00      0.00        97
#     Positive       0.58      1.00      0.73       164

#    micro avg       0.58      0.58      0.58       284
#    macro avg       0.19      0.33      0.24       284
# weighted avg       0.33      0.58      0.42       284

# {'loss': 1.02654, 'eval_accuracy': 0.5955882, 'eval_loss': 0.8804489, 'global_step': 465}
# [[  0   0  23]
#  [  0   0  97]
#  [  0   0 164]]
#               precision    recall  f1-score   support

#     Negative       0.00      0.00      0.00        23
#      Neutral       0.00      0.00      0.00        97
#     Positive       0.58      1.00      0.73       164

#    micro avg       0.58      0.58      0.58       284
#    macro avg       0.19      0.33      0.24       284
# weighted avg       0.33      0.58      0.42       284

# SAME FOR 10 EPOCHS!!!!!

In [0]:
predictions

In [0]:
len(data_train)

22284

In [0]:
len(data_train['DOCUMENT'])

22284